In [2]:
import casadi as ca
import numpy as np
from mpc_model import Model

ref: 
1. An augmented lagrangian based algorithm for distributed nonconvex optimization
2. Distributed Optimization with ALADIN for Non-convex Optimal Control Problems

In [29]:
def integrator_stage_cost(f, l, t, x, xr, u, ur, lambda_, delta_t):
    '''
    This function calculates the integration of stage cost with RK4.
    '''

    k1 = f(t, x, u)
    k2 = f(t + delta_t / 2, x + delta_t / 2 * k1, u)
    k3 = f(t + delta_t / 2, x + delta_t / 2 * k2, u)
    k4 = f(t + delta_t, x + delta_t * k3, u)

    Q = 0
    k1_q = l(x, xr, u, ur, lambda_)
    k2_q = l(x + delta_t / 2 * k1, xr, u, ur, lambda_)
    k3_q = l(x + delta_t / 2 * k2, xr, u, ur, lambda_)
    k4_q = l(x + delta_t * k3, xr, u, ur, lambda_)
    Q = Q + delta_t / 6 * (k1_q + 2 * k2_q + 2 * k3_q + k4_q)
    return Q

In [30]:
def integrator_rk4(f, t, x, u, delta_t):
    """
    Runge-Kutta 4th order solver using casadi.

    Args:
        f: First order ODE in casadi function (Nx + Nt -> Nx).
        t: Current time.
        x: Current value.
        u: Current input.
        delta_t: Step length.
    Returns:
        x_next: Vector of next value in casadi DM
    """
    k1 = f(t, x, u)
    k2 = f(t + delta_t / 2, x + delta_t / 2 * k1, u)
    k3 = f(t + delta_t / 2, x + delta_t / 2 * k2, u)
    k4 = f(t + delta_t, x + delta_t * k3, u)
    x_next = x + delta_t / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

    return x_next

In [31]:
def ode(t, x, u):
    """
    Mobile robot
    """
    # Parameter configuratio

    dx1_dt = x[1]
    dx2_dt = u[0] 
    rhs = [dx1_dt,
           dx2_dt
           ]
    return ca.vertcat(*rhs)

### Exmaple
$\begin{aligned} \min _{u_{i}, i \in \mathcal{V}} & \sum_{i \in \mathcal{V}} \int_{0}^{T}-e^{\left(-x_{1, i}^{2}\right)}-e^{\left(-x_{2, i}^{2}\right)}+\frac{1}{2} u_{i}^{2} \mathrm{~d} \tau \\ \text { s.t. } &\left[\begin{array}{l}\dot{x}_{1, i} \\ \dot{x}_{2, i}\end{array}\right]=\left[\begin{array}{c}x_{2, i} \\ u_{i}\end{array}\right] \\ & 1=\sum_{i \in \mathcal{V}} x_{1, i}+x_{2, i}+u_{i} \end{aligned}$

With $T = 1s$ and $N = |\mathcal{V}|=10$.

In [32]:
b = 1
Ci = np.array([[1, 1]])
Di = 1
rho = 1

Sigma_xi = ca.diag([1,1])
Sigma_ui = ca.diag([1])

In [33]:
Nt = 1
N_lambda, Nx = np.shape(Ci)
Nu = 1


delta_t = 0.1
N_pred = 10
# N_sim = 10

t_SX = ca.SX.sym("t_SX", Nt)
x_SX = ca.SX.sym("x_SX", Nx)
u_SX = ca.SX.sym("u_SX", Nu)
lambda_SX = ca.SX.sym("lambda_SX", N_lambda)

ode(t_SX, x_SX, u_SX)

xr_SX = ca.SX.sym("xr_SX", Nx)
ur_SX = ca.SX.sym("ur_SX", Nu)

# Cost function to calculate the local solution
stage_cost =  -ca.exp(-x_SX[0] **2) -ca.exp(-x_SX[1] **2) + 1/2 * u_SX[0]**2  #  Lagrange term
stage_cost += lambda_SX.T @ (Ci @ x_SX + Di @ u_SX) + rho / 2 * (x_SX - xr_SX).T @ Sigma_xi @ (x_SX - xr_SX) + rho / 2 * (u_SX - ur_SX).T @ Sigma_ui @ (u_SX - ur_SX)
terminal_cost = 0   #  Mayer term
stage_cost_func = ca.Function("stage_cost_func",[x_SX, xr_SX, u_SX, ur_SX, lambda_SX], [stage_cost])
terminal_cost_func = ca.Function("terminal_cost_func",[x_SX, xr_SX], [terminal_cost])

ode_func = ca.Function("ode_func", [t_SX, x_SX, u_SX], [ode(t_SX, x_SX, u_SX)])
ode_dis_model = integrator_rk4(ode_func, t_SX, x_SX, u_SX, delta_t)
ode_dis_model_func = ca.Function("ode_dis_model_func",[t_SX, x_SX, u_SX],[ode_dis_model])
stage_cost_dis = integrator_stage_cost(ode_func, stage_cost_func, t_SX, x_SX, xr_SX, u_SX, ur_SX, lambda_SX, delta_t)
stage_cost_dis_func = ca.Function("stage_cost_dis_func",[x_SX, xr_SX, u_SX, ur_SX, lambda_SX],[stage_cost_dis])

### Example 1
$\min _{x} x_{1} \cdot x_{2} \quad$ s.t. $\quad x_{1}-x_{2}=0$

analytical solution of $y$ is: $y=\left(\begin{array}{c}-2 \\ 2\end{array}\right) \lambda$

Numerical problem of ADMM with divergent $\lambda^{+} = - 2 \lambda$

### Subproblem formulation
$\min _{y_{i}} f_{i}\left(y_{i}\right)+\lambda^{\top} A_{i} y_{i}+\frac{\rho}{2}\left\|y_{i}-x_{i}\right\|_{\Sigma_{i}}^{2} \quad$ s.t. $\quad h_{i}\left(y_{i}\right) \leq 0 \mid \kappa_{i}$

In [35]:
def create_subproblem(fi_func, Ai, rho, hi_func):
    N_lambda, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    sigma_i = ca.SX.sym('sigma_i',N_yi,N_yi)
    lambda_ = ca.SX.sym("lambda",N_lambda)
    
    fi = fi_func(yi) + lambda_i.T @ Ai @ yi + rho/2 * (yi - xi).T @ sigma_i @ (yi - xi)
    p = ca.vertcat(lambda_, xi, sigma_i.reshape(-1,1))
    g = hi_func(yi)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [63]:
N =3
zero_row = ca.DM.zeros(1,3)


A = ca.DM([[1,2,3],[4,5,6],[7,8,9]])
A[1,:] = zero_row
A

DM(
[[1, 2, 3], 
 [0, 0, 0], 
 [7, 8, 9]])

In [77]:
x = ca.SX.sym("x",3)
hi = ca.vertcat(x[1]+x[2],x[0]+x[2])
hi_func = ca.Function("hi_func",[x],[hi])
hi_jac = ca.jacobian(hi,x)
hi_jac_func = ca.Function('hi_jac_func',[x], [hi_jac])

y = ca.DM([1,-1,1])
hi_jac_approx = constraint_jac_approx(y,hi_func,hi_jac_func)
hi_jac_real = hi_jac_func(y)

print(hi_jac_approx,hi_jac_real)


[[00, 1, 1], 
 [0, 0, 0]] 
[[00, 1, 1], 
 [1, 00, 1]]


In [72]:
def constraint_jac_approx(yi, hi_func, hi_jac_func):
    constraint_res = hi_func(yi)    #  Residue
    Nh = np.shape(constraint_res)[0]
    Ny = np.shape(yi)[0]
    zero_row = ca.DM.zeros(1,Ny)
    hi_jac = hi_jac_func(yi)
    for i in range(Nh):
        if constraint_res[i] != 0:    #  TODO: deal with small value
            hi_jac[i,:] = zero_row
    return hi_jac

In [78]:
def modified_grad(yi, fi_grad, hi_jac_approx, hi_jac_real, kappa_i):
    return fi_grad + (hi_jac_real - hi_jac_approx) @ kappa_i
    

In [83]:
print(ca.reshape(hi_jac_real,-1,1).shape)

(6, 1)


In [85]:
def create_QP_problem(A_list, b,  mu, N_hi_list):
    N = len(A_list)
    N_lambda = ca.shape(A_list[0])[0]
    
    s = ca.SX.sym("s", N_lambda)
    lambda_ = ca.SX.sym("lambda_", N_lambda)
    
    delta_yi_list = []
    fkh_hess_col_list = [] 
    modiefied_grad_col_list = []
    Ci_col_list = []
    
    yi_list = []
    obj = 0
    sigma_Ai = 0

    g = []
    for i in range(N):
        Ai = A_list[i]
        N_delta_yi = np.shape(Ai)[1]
        Hi = ca.SX.sym("Hi", N_delta_yi, N_delta_yi)
        gi = ca.SX.sym("gi", N_delta_yi)
        yi = ca.SX.sym("yi", N_delta_yi)
        Ci = ca.SX.sym("Ci", N_hi_list[i], N_delta_yi)
        
        fkh_hess_col_list += [Hi.reshape(-1,1)]
        modiefied_grad_col_list += [gi.reshape(-1,1)]
        yi_list += [yi]
        
        delta_yi = ca.SX.sym("delta_yi",N_delta_yi)
        delta_yi_list += [delta_yi]
    
        obj += 1/2 * delta_yi.T @ Hi @ delta_yi + gi.T @ delta_yi
        sigma_Ai += Ai @ (yi + delta_yi)
        
        Ci_col_list += [Ci.reshape(-1,1)]
        g += [Ci @ delta_yi]
    obj += lambda_.T @ s + mu/2 * s.T @ s
    x = ca.vertcat(*delta_yi_list, s)
    p = ca.vertcat(*(fkh_hess_col_list + modiefied_grad_col_list + yi_list + Ci_col_list))
    

    g += [ sigma_Ai - b - s ]
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

In [ ]:
eps = 1e-5
rho = 0.75

N_itermax = 50
A_list = []
sigma_list = []
fi_func_list = []
hi_func_list = []
N_hi_list = []

A = ca.DM([[1,-1]])
A_list += [A]
N = len(A_list)
b = ca.DM([0])

sigma_i = ca.diag([1,1])
sigma_list += sigma_i

Nx = 2
x = ca.SX.sym("x",Nx)
fi = x[0] * x[1]
fi_func = ca.Function("fi_func", [x], [fi])
fi_func_list += [fi_func]

fi_grad = ca.gradient(fi, x)
fi_grad_func = ca.Function("fi_grad_func", [x], [fi_grad])
fi_grad_func_list += [fi_grad_func]


hi = 0 # To be modified
hi_func = ca.Function("hi_func", [x], [hi])
Nhi = np.shape(hi)[0]
N_hi_list = [Nhi]
hi_func_list = [hi_func]

kappa_i = ca.SX.sym("kappa_i",Nhi)


hi_jac_func_list = []
fkh_hess_func_list = []
hi_func = ca.Function("hi_func",[x],[hi])
hi_jac = ca.jacobian(hi,x)
hi_jac_func = ca.Function("hi_jac_func",[x],[hi_jac])
hi_jac_func_list +=  hi_jac_func
fkh_i = fi + kappa_i @ hi
fkh_hess_i = ca.hessian(fkh, x)
fkh_hess_i_func = ca.Function("fkh_hess_i_func", [x, kappa_i], [fkh_hess_i])
fkh_hess_func_list += [fkh_hess_i_func]

subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    sigma_i = sigma_list[i]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list
    subsolver_list += [create_subproblem(fi_func, Ai, rho, sigma_i, hi_func)]    
mu = 1e5
QP_solver = create_QP_problem(A_list, b,  mu, N_hi_list)
# Initial guess
delta_yi_list = []
sigma_i_list = []
yi_list = []
lbhi_list = []
ubhi_list = []
Nx = 0
for i in range(N):
    Ai = A_list[i]
    N_lambda, N_xi = np.shape(Ai)
    Nx += N_xi
    N_hi = N_hi_list[i]
    xi = np.random.randn(N_xi,1).flatten().tolist()
    xi_list += [xi]
    
    sigma_i_list += [ca.diag([1] * N_xi)]
    
    lbhi_list += [[-ca.inf] * N_hi]
    ubhi_list += [[0] * N_hi]
    yi_list += [[0] * N_xi]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
lambda_ = np.random.randn(N_lambda,1)
s_list = [0] * N_lambda
delta_yi_list = sum(yi_list,[])
    
nl_sub = {}
nl_sub['lbg'] = lbhi_list
nl_sub['ubg'] = ubhi_list


nl_qp = {}
nl_qp['lbg'] = [0] * N_lambda
nl_qp['ubg'] = [0] * N_lambda
nl_qp['lbx'] = sum(lbx_list,[])
nl_qp['ubx'] = sum(ubx_list,[])

# Track solution
yi_sol_list = []
delta_y_sol_list = []
lambda_list = []
s_list = []
xi_list += xi_list
# delta_y_sol_list += [sum(xi_list,[])]
lambda_list += lambda_.flatten().tolist()
for i in range(N_itermax):
    sum_Ay = 0
    kappa_list = []
    # Step 1: solve the subproblem      
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        sigma_i = sigma_i_list[j]
      
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_list + xi_list[j] + sigma_i.reshape(-1,1).full().tolist()

        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
        yi_sol_list += [yi_list[j]]

        kappa_i = yi_sol['lam_g']
        kappa_list += [kappa_i]
        
        sum_Ay += Ai @ yi_sol['x']
#         lambda_i_list[j] = (ca.DM(lambda_i_list[j]) + rho * Ai @ (ca.DM(yi_list[j]) - ca.DM(xi_list[j]))).full().flatten().tolist()
#         lambda_i_sol_list += [lambda_i_list[j]]
    # Step 2: Check if the tolerance satisfied
    if ca.norm_1(sum_Ay - b) <= eps:
        break
    for j in range(N):
        if rho * ca.norm_1( sigma_i_list[j] @ ca.DM(yi_list[j])) <= epsilon:
            break
    # Step3: update Jacobian approximations, calculate the modified gradient, and update Hessian
    Ci_list = []    #  constraint Jacobian
    g_list = []    #  modified gradient
    H_list = []    #  Hessian
    for j in range(N):
        # 3.1 Choose Jacobian approximations
        yi = yi_list[j]
        hi_func = hi_func_list[j]
        hi_jac_func = hi_jac_func_list[j]
        fkh_hess_func = fkh_hess_func_list[j]
        hi = hi_func(yi)
        kappa_i = kappa_list[j]
        fi_grad = fi_grad_func_list[j](yi)
        hi_jac_real = hi_jac_func(yi)
        
        hi_jac_approx = constraint_jac_approx(yi, hi_func, hi_jac_func)
        Ci_list += [hi_jac_real.reshape(-1,1)]
        gi = modified_grad(yi, fi_grad, hi_jac_approx, hi_jac_real, kappa_i)
        g_list += [gi.reshape(-1,1)]
        
        Hi = fkh_hess_func(x, kappa_i)
        H_list += [Hi.reshape(-1,1)]
    # Step 4: Solve QP problem
    nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess
    
    H_para = ca.vertcat(*H_list)
    modiefied_grad = ca.vertcat(*g_list)
    y = ca.vertcat(* sum(yi_list,[]))
    Ci = ca.vertcat(*Ci_list)
    p = ca.vertcat(H_para, modiefied_grad, y, Ci)
    nl_qp['p'] = p
    QP_sol = solver_subproblem(**nl_qp)
    
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    # Step 5: Update x and lambda
    pos = 0
    QP_list = deltay_sol['x'].full().flatten().tolist()
    x_plus = ca.DM(sum(xi_list,[])) + alpha1 * ()
    for j in range(N):
        list_len = len(xi_list[j])
        xi_list[i] = xi_plus_list[pos:pos+list_len]
    
    x_plus_sol_list += [xi_plus_list]